In [ ]:
import time,serial,random
import numpy as np
import scipy,pybrain3

def servoin(a=90,b=90,c=90,d=90,e=90,f=90,delay=1):
    '''waits end of previous act. and send angles to arduino returns acc-array     
    '''
    while ser.read()!=b'>':
        pass
    acc = ser.readline().split(b'\t')
    A=np.array([[float(acc[0])],[float(acc[1])],[float(acc[2])]])
    print(*A)
    if fall_check(*A)==False:
        data='in'
        for i in a,b,c,d,e,f,delay:
            try: 
                data+=str(hex(i))[2]+str(hex(i))[3]
            except IndexError:
                data+='0'+str(hex(i))[2]
        ser.write(data.encode())
        print(data)
    else:
        print('I have fallen')
        1/0

    
def fall_check(accelx,accely,accelz):
    '''gets measurement of accelerometer
        checks if Robo has fallen
        returns False if Robo - NOT fallen
        returns True if Robo - has fallen'''
    if (abs(accelx)>5 or abs(accelz-2)>5) and accely<7:
        return True
    else: return False
    
def interpreter(st,mtrx):
    '''gets input string and last value of servoin
        returns tuple of updated values servo
        if str='start' to initials
        if str='a=12 c=13' returns (12,90,13,90,90,90,1)
        if str='(12,90,13,90,90,90,1)' returns the same
        if str='(12,90,13,90)' returns the same
        if str='a=12 c=13 then e=50 then f=40 then start' compile the algorythm
        if str='a=5 c=3 then while a=6 b=2 and a=4 b=4' compile
        if str='a=5 c=3 then wait 3 then b=5' compile
    '''
    old=[*mtrx[-1]]
    if st=='start':
        mtrx.append([90,90,90,90,90,90,1])
        return mtrx
    
    if st=='jackson':
        mtrx = interpreter('start then for 8 b=120 d=120 and a=120 c=120 and wait 1 and a=60 c=60 and b=60 d=60',mtrx)
        return mtrx
    
    if st=='walk':
        mtrx = interpreter('start then for 8 e=80 g=3 and f=80 b=70 a=70 and a=50 and d=70 f=90 e=90 c=70 and e=110 f=110 b=100 a=90 d=90 and c=100 e=120 b=120 g=1 and b=130 d=110 a=120 c=120 and c=140 e=90 f=90 g=3 and e=80 f=80 d=80 b=90 a=60 c=80',mtrx)
        return mtrx
    
    if st[0:4]=='wait':
        time.sleep(int(st[5]))
        mtrx=interpreter(st[11:],mtrx)
        return mtrx
    
    if st[0:5]=='while':
        while True:
            for j in st[6:].split(' and '):
                mtrx=interpreter(j,mtrx)
    
    if st[0:3]=='for':
        for l in range(int(st[4])): 
            for j in st[6:].split(' and '):
                mtrx=interpreter(j,mtrx)
        return mtrx
    
    try: #12,13,15
        q = list(int(i) for  i in st.split(','))
        for j in range(len(q),7): q.append(old[j])
        old = [q[k] for k in range(len(q))]
        mtrx.append(old)
        return mtrx
    except ValueError:
        try: #a=12 b=13
            q = old
            tmp = list(st.split())
            for i in tmp: 
                q[ord(str(i).split('=')[0])-97]=int(str(i).split('=')[1])
            old = [q[k] for k in range(len(q))]
            #print(old)
            mtrx.append(old)
            return mtrx
        except IndexError: #a=12 b=13 then b=15
            for j in st.split(' then '):
                mtrx=interpreter(str(j),mtrx)
            return mtrx
        
def serial_begin(port):
    ser = serial.Serial(port, 19200, bytesize=8, parity='N', stopbits=1, timeout=2)
    time.sleep(2)
    print("connected to: "+ ser.portstr)
    return ser,[[90,90,90,90,90,90,1]]

if __name__ == "__main__":
    port = 'COM5'
    ser,matrix=serial_begin(port)
    while True:
        try:
            inp=input()
            matrix=interpreter(inp,[[*matrix[-1]]])
            print(matrix)
            for i in matrix: servoin(*i)
        except ZeroDivisionError:
            ser.close()
            ser,matrix=serial_begin(port)
    ser.close()


connected to: COM5
walk
[[90, 90, 90, 90, 90, 90, 1], [90, 90, 90, 90, 90, 90, 1], [90, 90, 90, 90, 80, 90, 3], [70, 70, 90, 90, 80, 80, 3], [50, 70, 90, 90, 80, 80, 3], [50, 70, 70, 70, 90, 90, 3], [90, 100, 70, 90, 110, 110, 3], [90, 120, 100, 90, 120, 110, 1], [120, 130, 120, 110, 120, 110, 1], [120, 130, 140, 110, 90, 90, 3], [60, 90, 80, 80, 80, 80, 3], [60, 90, 80, 80, 80, 80, 3], [70, 70, 80, 80, 80, 80, 3], [50, 70, 80, 80, 80, 80, 3], [50, 70, 70, 70, 90, 90, 3], [90, 100, 70, 90, 110, 110, 3], [90, 120, 100, 90, 120, 110, 1], [120, 130, 120, 110, 120, 110, 1], [120, 130, 140, 110, 90, 90, 3], [60, 90, 80, 80, 80, 80, 3], [60, 90, 80, 80, 80, 80, 3], [70, 70, 80, 80, 80, 80, 3], [50, 70, 80, 80, 80, 80, 3], [50, 70, 70, 70, 90, 90, 3], [90, 100, 70, 90, 110, 110, 3], [90, 120, 100, 90, 120, 110, 1], [120, 130, 120, 110, 120, 110, 1], [120, 130, 140, 110, 90, 90, 3], [60, 90, 80, 80, 80, 80, 3], [60, 90, 80, 80, 80, 80, 3], [70, 70, 80, 80, 80, 80, 3], [50, 70, 80, 80, 80, 80, 

[0.16] [9.96] [2.24]
in5a5a5a5a5a5a01
[0.27] [10.04] [2.24]
in5a5a5a5a505a03
[0.43] [9.77] [2.28]
in46465a5a505003
[0.67] [5.06] [4.08]
in32465a5a505003
[0.75] [10.36] [-0.24]
in324646465a5a03
[3.8] [9.02] [-0.2]
in5a64465a6e6e03
[2.28] [7.85] [2.47]
in5a78645a786e01
[3.73] [12.55] [0.86]
in7882786e786e01
[5.14] [7.73] [2.51]
in78828c6e5a5a03
[0.2] [9.41] [4.79]
in3c5a5050505003
[0.12] [8.83] [5.22]
in3c5a5050505003
[-0.78] [9.3] [4.28]
in46465050505003
[-0.35] [8.28] [4.12]
in32465050505003
[0.75] [10.32] [0.9]
in324646465a5a03
[-6.86] [10.75] [1.96]
in5a64465a6e6e03
[2.12] [5.49] [4.82]
in5a78645a786e01
[3.14] [17.77] [-4.55]
in7882786e786e01
[0.2] [6.2] [4.47]
in78828c6e5a5a03
[8.79] [16.44] [-5.88]
in3c5a5050505003
[-0.27] [9.18] [3.3]
in3c5a5050505003
[-1.33] [9.41] [4.16]
in46465050505003
[-1.02] [8.87] [2.67]
in32465050505003
[-0.43] [9.53] [3.06]
in324646465a5a03
[-1.22] [9.26] [3.3]
in5a64465a6e6e03
[-1.14] [6.75] [4.75]
in5a78645a786e01
[-2.9] [9.3] [5.18]
in7882786e786e01
[-

In [ ]:
ser.close()